In [3]:
# !pip install chromadb faiss-cpu

In [1]:
import urllib.request as request
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader, PDFPlumberLoader
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [3]:
request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf",
                    "data-files/pdf-example-01.pdf")

('data-files/pdf-example-01.pdf', <http.client.HTTPMessage at 0x271b811c7c0>)

In [4]:
%%time
# pdf 파일의 텍스트 정보를 읽어서 페이지 단위로 반환
loader = PyPDFLoader('data-files/pdf-example-01.pdf')
pages = loader.load_and_split()

CPU times: total: 14.4 s
Wall time: 14.7 s


In [17]:
# print( pages[0] )
print(len(pages))
pages[99].page_content

445


'2023 북한인권보고서\n100\n소이다. ‘구금소’는 각 도의 보위국 이상에서만 운영하는 시설로, 주\n로 정치범(한국 관련)과 연관된 범죄 혐의자를 구금하는 곳으로 보\n인다. 보위부의 예심으로 이관될지 여부가 결정되기 전까지 구금되\n는 곳이라는 진술이 있었다. \n형집행시설인 교화소와 노동단련대는 사회안전성에서 운영하는\n데, ‘교화소’는 형법에 명시된 형집행 시설로 수사-예심-재판의 형\n사소송절차를 거쳐 노동교화형이 확정된 자를 수용하는 곳이다. ‘노\n동단련대’는 재판으로 노동단련형이 확정된 자를 구금하는 시설이\n다. 북한 형법에서도 노동단련형의 형집행 장소를 노동단련대로 명\n시하고 있으며,74 교화소에서 부속시설로 노동단련대를 운영하기도 \n하는 것으로 보인다. ‘노동교양대’는 북한 행정처벌법(2020)에 규정\n된 ‘노동교양처벌’을 집행하는 시설이다.75 노동교양대는 지역은 물\n론 비교적 규모가 큰 직장이나 농장에도 설치되어 있는 것으로 보이\n는데, 북한 주민들은 노동교양대와 노동단련대라는 명칭을 혼용하\n고 있었다. \n이외에도 각종 단속을 위해 안전부, 보위부, 검찰소, 청년동맹 \n등이 연합하여 조직한 일명 ‘연합지휘부’의 시설에 구금되는 사례\n도 상당수 수집되었다. 이 경우 구금 장소가 일정하지 않고 조직의 \n사무공간 등 임의의 공간인 경우가 대부분이었다. 인민보안단속법\n(2007)에서는 이와 같은 시설에 감금하는 것을 ‘억류’라고 지칭하고 \n있다.76 \n74\t \t「형법(2022)」\t제39조(노동단련형)\t노동단련형은\t범죄자를\t노동단련대에\t보내여\t노동을\t시키는\t형벌이다.\n75\t \t「행정처벌법(2021)」\t제18조\t노동교양처벌은\t무거운\t위법행위를\t한\t공민을\t노동교양대에\t보내여\t노동을\t\n시키는\t행정법적\t제재이다.(이하\t생략)\n76\t \t「인민보안단속법(2007)」\t제47조(억류대상)\t인민보안원은\t법질서를\t어기고\t도주하거나\t또는\t공모하여\t법'

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [19]:
documents = text_splitter.split_documents(pages)

In [20]:
len(documents)

497

In [25]:
len( documents[122].page_content )

1000

In [26]:
# 분할된 문서들을 vectordb에 저장 (여기서는 Chroma)

# chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings()) # 메모리에 저장
chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings(), 
                                  persist_directory="vectordb/chroma.db") # 파일로 저장

C:\Users\human\AppData\Local\Temp\ipykernel_18640\453854206.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings(),


In [27]:
chroma_db2 = Chroma(persist_directory="vectordb/chroma.db", 
                    embedding_function=OpenAIEmbeddings())

C:\Users\human\AppData\Local\Temp\ipykernel_18640\1059780125.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db2 = Chroma(persist_directory="vectordb/chroma.db",


In [28]:
chroma_db._collection.count(), chroma_db2._collection.count()

(994, 994)

In [29]:
searched_documents = chroma_db2.similarity_search("북한의 교육과정")

In [35]:
print(len(searched_documents))
print( searched_documents[3].page_content )

4
2023 북한인권보고서
342
2018년에 학교에서 추천하여 소년궁전 스키부에 선발되었으나, 체
육종합지도원이 자신의 출신성분이 좋지 않다는 이유로 선발명단에
서 자신을 제외했다고 진술하였다. 정치범수용소에서는 이주민 자
녀의 경우 정규교육과정을 받지 못한다는 증언도 있었다. 정치범수
용소에도 소학교와 중학교가 있지만 일반 학교와는 달리 학생들이 
책가방 대신 지게를 지고 출석하고 학교에서 농사짓는 법에 대해 배
운다고 한다. 
북한에서는 대학 진학 시에도 출신성분에 따른 차별이 존재하는 
것으로 나타났다. 한 증언자는 2019년에 고급중학교를 졸업하고 대
학에 진학하려고 했는데, 본인 문건에 어머니가 행방불명으로 되어 
있어 출신성분이 나쁘다는 이유로 대학에 진학할 수 없었다고 한다. 
이러한 차별행위는 교육부의 정책에 따른 것이라고 한다. 다른 증
언자는 2018년에 의학대학에 진학하고 싶었는데, 교육과장과 면담
하는 과정에서 ‘어머니가 행방불명이라 너 같은 아이는 대학에 가기 
어렵다.’는 이야기를 듣고 대학 진학을 포기했다고 한다. 중학교 졸
업 후 농민의 자식이라는 이유로 일반 대학뿐만 아니라 공업전문학
교도 진학할 수 없었다는 증언도 있었다. 
 
(2) 경제력에 따른 차별 
북한에서는 교육기회에 있어 경제력에 따른 차별이 존재하는 것
으로 보인다. 수집된 증언에 따르면 무상교육제가 유명무실해진 상
황에서 교육비 부담으로 경제적 취약계층의 아동들이 장기간 결석하
거나 중도에 학교를 그만두는 것으로 나타났다. 한 증언자는 2018년
에 아들이 소학교 학생이었는데, 당시 학급 정원의 25% 정도가 경제


In [24]:
faiss_db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [26]:
searched_documents2 = faiss_db.similarity_search("북한의 교육 과정")

In [32]:
print( searched_documents2[0].page_content )

2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또
한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실
탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 
하고 있다.
북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 
연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제
도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로


In [33]:
faiss_db.save_local('vectordb/faiss.db')

In [34]:
faiss_db2 = FAISS.load_local('vectordb/faiss.db',
                             OpenAIEmbeddings(),
                             allow_dangerous_deserialization=True)